In [1]:
import pandas as pd

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import cornac
from cornac.eval_methods import RatioSplit, BaseMethod
from cornac.models import WMF

import tensorflow as tf

print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

SEED = 42
VERBOSE = False

F:\Anaconda2\envs\Machinelearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cornac version: 1.6.0
Tensorflow version: 1.15.0


In [2]:
df = pd.read_csv('final_review_pa.csv',encoding='latin1')
print(df.head())

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  8JFGBuHMoiNDyfcxuWNtrA  smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ   
2  Xs8Z8lmKkosqW5mw_sVAoA  IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg   
3  J-4NdnDZ0pUQaUEEwDI9KQ  vrKkXsozqqecF3CW4cGaVQ  rjuWz_AD3WfXJc03AhIO_w   
4  G_5UczbCBJriUAbxz3J7Tw  clWLI5OZP2ad25ugMVI8gg  x4XdNhp0Xn8lOivzc77J-g   

   stars  useful  funny  cool  \
0    3.0       0      0     0   
1    4.0       0      0     0   
2    5.0       0      0     0   
3    5.0       2      2     2   
4    5.0       0      0     0   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  Good food--loved the gnocchi with marinara\nth...  2009-10-14 19:57:14  
2  My absolute favorite cafe in the city. Their b...  2014-11-12 15:30:27  
3  I thoroughly enjoyed the show. 

In [3]:
df2=pd.read_csv('business.csv',encoding='latin1')
df2.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [4]:
df3=pd.merge(df, df2, on='business_id')
print(df3.groupby('state').size())

state
PA    588270
dtype: int64


In [6]:
df3['date']=pd.to_datetime(df3['date'])
df3=df3.sort_values(by="date")
df4=df3[df3['state']=='PA'][['user_id','name','stars_x']].reset_index(drop=True)
splitindex=int(len(list(df4['user_id']))*0.7)
print(splitindex)
train_ratings=df4.iloc[:splitindex,:].reset_index(drop=True)
test_ratings=df4.iloc[splitindex:,:].reset_index(drop=True)

411789


In [29]:
wmf1 = WMF(k=50, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={50})")
wmf2 = WMF(k=70, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={70})")
wmf3 = WMF(k=90, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={90})")
wmf4 = WMF(k=110, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={110})")
wmf5 = WMF(k=130, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={130})")
wmf6 = WMF(k=150, max_iter=100, a=1.0, b=0.01, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,
          verbose=VERBOSE, seed=SEED, name=f"WMF(K={150})")

eval_metrics = [
  cornac.metrics.RMSE(), 
  cornac.metrics.AUC(),
  cornac.metrics.Precision(k=10),
  cornac.metrics.Recall(k=10),
  cornac.metrics.FMeasure(k=10),
  cornac.metrics.NDCG(k=[10, 20, 30]),
  cornac.metrics.MRR(),
  cornac.metrics.MAP()
]

base_method=BaseMethod.from_splits(
    train_ratings.to_records(index=False),
    test_ratings.to_records(index=False),
    rating_threshold=4.0,
    exclude_unknowns=False,
    verbose=VERBOSE,
    seed=SEED
)

cornac.Experiment(eval_method=base_method, models=[wmf1,wmf2,wmf3,wmf4,wmf5,wmf6], metrics=eval_metrics).run()

ERROR! Session/line number was not unique in database. History logging moved to new session 671


F:\Anaconda2\envs\Machinelearning\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 17600 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
F:\Anaconda2\envs\Machinelearning\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 7911 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)



TEST:
...
           |   RMSE |    AUC |  F1@10 |    MAP |    MRR | NDCG@10 | NDCG@20 | NDCG@30 | Precision@10 | Recall@10 | Train (s) | Test (s)
---------- + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------- + ------------ + --------- + --------- + --------
WMF(K=50)  | 2.3104 | 0.5325 | 0.0145 | 0.0143 | 0.0512 |  0.0217 |  0.0260 |  0.0298 |       0.0137 |    0.0240 | 1492.5668 | 142.7046
WMF(K=70)  | 2.3238 | 0.5252 | 0.0153 | 0.0154 | 0.0542 |  0.0235 |  0.0280 |  0.0316 |       0.0142 |    0.0265 | 1540.1369 | 144.8059
WMF(K=90)  | 2.3323 | 0.5177 | 0.0152 | 0.0156 | 0.0537 |  0.0235 |  0.0282 |  0.0321 |       0.0139 |    0.0267 | 1520.5920 | 125.8480
WMF(K=110) | 2.3416 | 0.5132 | 0.0158 | 0.0159 | 0.0542 |  0.0242 |  0.0291 |  0.0330 |       0.0144 |    0.0281 | 1555.6768 | 126.2943
WMF(K=130) | 2.3501 | 0.5095 | 0.0159 | 0.0162 | 0.0548 |  0.0246 |  0.0297 |  0.0334 |       0.0145 |    0.0283 | 1645.4923 | 128.0047
WMF(K=150) | 2.3573 | 0.5050 | 0.0158

In [8]:
print(df4['user_id'].nunique())
print(df4['name'].nunique())
print(df4.info())

26763
26337
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588270 entries, 0 to 588269
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  588270 non-null  object 
 1   name     588270 non-null  object 
 2   stars_x  588270 non-null  float64
dtypes: float64(1), object(2)
memory usage: 13.5+ MB
None
